In [17]:
import sort_RAFT_table as srt
import pandas as pd
import numpy as np
import plotly.express as px
import re
from scipy.optimize import curve_fit
import itertools

In [18]:
# ToDo: Kinetik Kurven erstellen,
#  90% oder max als max gemessen als conversion, als Beschreibung für diese nehmen,
#  in ist Monomer und Mn dann
#       sortieren nach Umsatz dahingehend dann nach kürzester Zeit
#       Tabellenausgabe dazu Kinetik Kurve, reaktionsbedingungen (LM, Raft agent, Zeit...)
#  am besten als interaktive website Tabelle.

# get all conversion values,
conversion_list = []
for column in srt.df.columns:
    if "conversion" in column:
        conversion_list.append(column)
print(conversion_list)

# get the hours from the headers with regex
hours_list = []
two_digit_regex = r"\d+"
for column in conversion_list:
    hours_list.append(int(re.findall(two_digit_regex, column)[0]))

print(hours_list)

['t0h-conversion', 't1h-conversion', 't2h-conversion', 't4h-conversion', 't8h-conversion', 't15h-conversion']
[0, 1, 2, 4, 8, 15]


In [19]:
def change_timeformat(time_format):
    h_m_s = str(time_format).split(":")
    m_format = int(h_m_s[0]) * 60 + int(h_m_s[1]) + int(h_m_s[2]) / 60
    return m_format

def change_timeformat_h(time_format):
    h_m_s = str(time_format).split(":")
    h_format = int(h_m_s[0]) + (int(h_m_s[1]) + int(h_m_s[2]) / 60) /60
    return h_format

# creating a table as a lookup to correct all sample measurement times
exact_times = pd.read_excel(srt.INPUT_FILE_PATH, sheet_name="exact sampling times")

time_correction_df = pd.DataFrame(data=exact_times.iloc[3:12, 3:])
time_correction_df.columns = exact_times.loc[2][3:]

time_correction_df.reset_index(inplace=True, drop=True)
ext_time_corr_df_data =[]
for row in time_correction_df.iterrows():
    reactor_nr = row[1]["Reactor"]
    if type(reactor_nr) == int or reactor_nr == "closing of reactors":
        row[1]["Reactor"] = str(reactor_nr)
        ext_time_corr_df_data.append(row[1])
    else:
        reactor_nr = (row[1]["Reactor"]).split("+")
        for nr in reactor_nr:
            row[1]["Reactor"] = nr
            ext_time_corr_df_data.append(row[1].copy())
ext_time_corr_df = pd.DataFrame(data=ext_time_corr_df_data, columns=exact_times.loc[2][3:])
ext_time_corr_df.reset_index(drop=True, inplace=True)
print("The timer was reset to 0 after the first closing of reactors in the \"t = 0\"-sampling-step")
ext_time_corr_df.columns = ["Reactor", 0, 1, 2, 4, 6, 8, 10, 15]

# change time format to minutes and set
time_cols = ext_time_corr_df.columns.difference(["Reactor"])
ext_time_corr_df[time_cols] = ext_time_corr_df[time_cols].apply(lambda x: [change_timeformat_h(d) for d in x])
ext_time_corr_df[0] = ext_time_corr_df[0].apply(lambda x: 0 )

# set index to reactor
ext_time_corr_df.set_index("Reactor", inplace=True)

# Sampling 6h and 10h were only used for sec not NMR -> no conversion for these.
skip_hours_mask = [True if h not in hours_list else False for h in ext_time_corr_df.columns]
ext_time_corr_df.drop(columns=time_cols[skip_hours_mask], inplace=True)

ext_time_corr_df



The timer was reset to 0 after the first closing of reactors in the "t = 0"-sampling-step


,0,1,2,4,8,15
Reactor,,,,,,
1,0,1.005556,2.011111,4.018056,8.016667,15.016667
2,0,1.005556,2.011111,4.018056,8.016667,15.016667
3,0,1.073611,2.068056,4.077778,8.090278,15.098611
4,0,1.073611,2.068056,4.077778,8.090278,15.098611
5,0,1.131944,2.125000,4.143056,8.180556,15.200000
6,0,1.131944,2.125000,4.143056,8.180556,15.200000
7,0,1.191667,2.184722,4.205556,8.256944,15.291667
8,0,1.191667,2.184722,4.205556,8.256944,15.291667
9,0,1.251389,2.241667,4.273611,8.338889,15.381944


In [20]:
# srt.df.columns
ext_time_corr_df.loc["15"]


0      0.000000
1      1.437500
2      2.430556
4      4.475000
8      8.581944
15    15.658333
Name: 15, dtype: float64

In [21]:
# plotting the kinetic curves of the RAFT polymerization
# generate a reformatted table with a entries per sample analysis taken described with a column of the right time

# create a dataframe with just the information needed to append to
kinetic_curves = []
for index, polymerisation_kinetic in srt.df.iterrows():
    # get the right hours list
    # polymerisation_kinetic["reactor"]

    kinetic_curve_entries = pd.DataFrame(
        {"conversion" : polymerisation_kinetic[conversion_list], "time" : hours_list, "reactor" : polymerisation_kinetic["reactor"]})
    kinetic_curve_entries["experiment"] = str(index)+"_"+polymerisation_kinetic["monomer"]+"_"+polymerisation_kinetic["RAFT-Agent"]+"_"+polymerisation_kinetic["solvent"]

    # the times are dependent on the current reactor, get current
    current_reactor_nr = str(polymerisation_kinetic["reactor"])
    current_time_list = ext_time_corr_df.loc[current_reactor_nr]
    kinetic_curve_entries["time"] = list(current_time_list)

    kinetic_curves.append(kinetic_curve_entries)

kinetic_curves_df = pd.concat(kinetic_curves)

In [22]:
len(kinetic_curves) # was 325 before getting away the average below 0.1, now its 317
# kinetic_curves[1]["experiment"].iloc[1]
# px.line(kinetic_curves_df[:20], x="time", y="conversion", color="experiment")

317

In [23]:
# and fit curve and the derivation
# find the maximum of the derivation

'''
deprecated: more theoretical accurate version but practically worse applicable
    # L is responsible for scaling the output range from [0,1] to [0,L]
    # b
    # k is responsible for scaling the input, which remains in (-inf,inf)
    # x0
def sigmoid (x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)


def sigmoid_derivative(x, L ,x0, k, b): # ToDo: check if this is the correct derivative
    y = (L*k*np.exp(-k*(x-x0)))/((np.exp(-k*(x-x0))+1)**2)
    return (y)
'''
def neg_growth(x, L, k):
    y = L * (1 - np.exp(k*(-x)))
    return y

def neg_growth_derivative(x, L, k):
    y = L * k * np.exp(k*(-x))
    return y

def linear_growth(x, m):
    y = m * x
    return y

def linear_growth_derivative(x, m):
    return m

xrange = np.arange(-0.2, 5, 0.1)
example_fig = px.line()
for testparams in ([1,1], [2,1], [1,2]):
    example_fig.add_scatter(x=xrange, y=neg_growth(xrange, *testparams))
    example_fig.add_scatter(x=xrange, y=neg_growth_derivative(xrange, *testparams), opacity=0.5, line=dict(dash="dot"))

print("visualisation of the functions")
# example_fig.show()

visualisation of the functions


In [113]:
# search for nans
for kinetic_curve in kinetic_curves:
    if kinetic_curve["conversion"].isnull().values.any():
        print(kinetic_curve["experiment"].iloc[1])
        print(kinetic_curve["conversion"].isnull().values)



4_7_A_DMSO
[False False False  True False False]
6_8_A_DMF
[False False False False  True  True]
11_10_A_DMF
[False False False  True False False]
18_12_A_DMSO
[False False False  True False False]
20_13_A_DMF
[False False False False  True False]
23_14_A_DMSO
[False False False False  True False]
24_15_A_DMF
[False  True False  True False False]
30_2_A_DMF
[False False False False False  True]
43_3_C_Tol
[False False False False False  True]
44_4_C_DMF
[False False False False False  True]
45_4_C_DMSO
[False False False False False  True]
46_4_C_Tol
[False False False False False  True]
47_5_C_Tol
[False False False False False  True]
55_10_C_DMF
[False False  True  True False False]
56_10_C_DMSO
[False False  True False False False]
57_10_C_Tol
[False False  True False False False]
59_11_C_DMSO
[False False False False False  True]
66_14_C_DMF
[False False False  True False False]
67_14_C_Tol
[False False False False  True  True]
87_3_B_DMF
[False False False False  True False]
98_7_

In [134]:
fig = px.line(title="Kinetic Curve Fit", labels={"x":"time", "y":"conversion"})
hard_to_fit = [] # array for kinetic fits that meet a filter criteria further down the code
colors = px.colors.qualitative.Plotly # set up a simple color palette
extended_xdata = np.linspace(-1, 16.5, 100) # x data array for

# create new dataframe with kinetics per row
kinetic_curve_columns = ["Experiment", "max_conversion", "reaction_end", "squared_error"]
kinetics_df = pd.DataFrame(columns=kinetic_curve_columns)

def add_fits_to_plot(fit_func, fit_func_derivative, fit_func_params, marker_dic):
    fig.add_scatter(
        x=extended_xdata, y=fit_func(extended_xdata, *fit_func_params),
        opacity=1, line=dict(dash="dot"), name=f"{fit_func.__name__} fit", marker=marker_dic)
    print(fit_func_params)
    fig.add_scatter(
        x=extended_xdata, y=fit_func_derivative(extended_xdata, *fit_func_params),
        opacity=0.3, line=dict(dash="dash"), name=f"{fit_func_derivative.__name__}", marker=marker_dic)

def fit_and_exclude_outliers(x, y, fit_func, p0, bounds, nan_policy="omit", iteration=1):
    cf_data = curve_fit(f=fit_func, xdata=x, ydata=y, p0=p0, nan_policy=nan_policy, maxfev=800 * 10, bounds=bounds)

    # exclude the nan values from the data
    mask = ~np.isnan(y)
    x, y = x[mask], y[mask]

    # calculate the fit points
    fit_points = np.array([fit_func(x, *cf_data[0]) for x in x])
    # calculate the standard deviation of the residuals between the fit and the datapoints
    sigma = np.std(fit_points - y)

    # exclude the outliers
    mask = ~(np.abs(fit_points - y) > 2 * sigma)
    if not mask.all():
        print(mask, sigma, y - fit_points)

    result = {"x": x[mask], "y": y[mask], "p_opt" : cf_data[0], "p_cov" : cf_data[1], "excluded_points" : (x[~mask],y[~mask]), "iteration" : iteration+1}
    return result

for idx, kinetic_curve in enumerate(kinetic_curves):
    # first make sure the datapoints are in the right format and not sometimes int sometimes float
    ydata = np.array(kinetic_curve["conversion"].values, dtype=float)
    xdata = np.array(kinetic_curve["time"].values, dtype=float)

    # choosing a few test function
    # if not idx in (316, 301, 302):
    #     continue

    # fitting section
    p0 = [max(ydata), 0.1] # this is a mandatory initial guess
    fit_dic = fit_and_exclude_outliers(x=xdata, y=ydata, fit_func=neg_growth, p0=p0, bounds=([0, -np.inf], [1, np.inf]))
        # f=neg_growth, xdata=xdata, ydata=ydata, p0=p0, nan_policy="omit", maxfev=800*10, bounds=([0, -np.inf], [1, np.inf]))


        # fit with linear growth
    p0_l = [max(ydata)/7]
    cv_data_l = curve_fit(f=linear_growth, xdata=xdata, ydata=ydata, p0=p0_l, nan_policy="omit")
    popt_l, pcov_l = cv_data_l

    popt, pcov = fit_dic["p_opt"], fit_dic["p_cov"]

    # calculate the squared error of fit and data points
    y_st =[]
    x_st = []
    for y_val, x_val in zip(ydata, xdata):
        if np.isnan(y_val) or np.isnan(x_val):
            continue
        y_st.append(y_val)
        x_st.append(x_val)
    y_st = np.array(y_st)
    x_st = np.array(x_st)
    squared_error = np.sum((y_st - neg_growth(x_st, *popt))**2)
    # print("{:.6f}".format(squared_error), str(idx) + ": The optimal parameters are " + " and ".join("%.3f" % x for x in popt) + ". The variance is " + " and respectively ".join("{:.3f} {:.3f}".format(x, y) for x, y in pcov) + ".")

    # ToDo: fill new dataframe
    new_row = pd.DataFrame({"Experiment":kinetic_curve["experiment"].iloc[1], "max_conversion":max(ydata),
                        "reaction_end":"yet to calc", "fit_p1":[popt[0]],"fit_p2":[popt[1]], "p1_err1":[pcov[0][0]],"p1_err2":[pcov[0][1]],"p2_err1":[pcov[1][0]],"p2_err2":[pcov[1][1]], "squared_error":squared_error})
    kinetics_df = pd.concat([kinetics_df, new_row])

    # filter for inspection
    # if not ((popt[1]) < 0.001).any(): continue
    # hard_to_fit.append(idx)

''' comment out if no plot is needed.
    marker_dict = dict(color=colors[idx%len(colors)]) # set same colors per kinetic
    fig.add_scatter(x=xdata, y=ydata, mode="lines+markers", opacity=1, name=kinetic_curve["experiment"].iloc[1], marker=marker_dict)
    add_fits_to_plot(neg_growth, neg_growth_derivative, popt, marker_dict)
    # add_fits_to_plot(linear_growth, linear_growth_derivative, [popt_l], marker_dict)


fig.update_layout(
    yaxis=dict(
        range=[-0.1,1]
    ),
    xaxis_title="Time [h]",
    yaxis_title="Conversion [%]"
)
fig.show()
# '''
print(hard_to_fit, len(hard_to_fit)/len(kinetic_curves))
kinetics_df.reset_index(drop=True, inplace=True)


[ True False  True  True  True  True] 0.013577628844622285 [ 0.          0.03063802 -0.00398808  0.00520153 -0.01374768  0.00472388]
[ True False  True  True  True  True] 0.00958005545655354 [ 0.          0.02281596  0.00067712 -0.0047017  -0.00620266  0.0034692 ]
[ True False  True  True  True  True] 0.005378877805722397 [ 0.          0.01161431 -0.00619461 -0.00153211 -0.00049797  0.00087105]
[ True False  True  True  True  True] 0.02505213870604597 [ 0.         -0.05626801  0.00210657  0.00906075  0.02343324 -0.01266272]
[ True  True False  True  True  True] 0.03789674577632354 [ 0.         -0.0119241  -0.07614567 -0.01667498 -0.03739086  0.04934954]
[ True  True  True  True  True False] 0.014261780966035063 [ 0.          0.00313508 -0.0102892  -0.01196574 -0.0100665   0.0293289 ]
[ True  True False  True  True  True] 0.041439331034110904 [ 0.         -0.04407177 -0.08536045 -0.00359037  0.03927723  0.02039112]
[ True False  True  True  True  True] 0.005464226884367509 [ 0.         

C:\Users\fo72qev\AppData\Local\Temp\ipykernel_3744\4019977708.py:77: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



[ True  True False  True  True  True] 0.004788804462985619 [ 0.         -0.00557741 -0.00986894  0.00439111 -0.00270659  0.00219081]
[ True  True  True False  True  True] 0.025569906161204848 [ 0.          0.00053469 -0.00269333 -0.05642825 -0.00727643  0.02989773]
[ True False  True  True  True  True] 0.013216174860373566 [ 0.         -0.03083203  0.00233511  0.00954037  0.00577572 -0.00429973]
[ True  True False False] 0.01103567135432128 [ 0.         -0.0130303  -0.02757576 -0.02545455]
[ True False  True  True  True  True] 0.007399971898801327 [ 0.         -0.01877549  0.00065501 -0.00486081  0.00312963  0.00105025]
[ True False  True  True  True  True] 0.00936737152420375 [ 0.         -0.01914635 -0.0074772   0.01111616 -0.0054788   0.00259915]
[ True  True False  True  True  True] 0.004289419765787752 [ 0.          0.00376516 -0.00954513  0.00213147 -0.00112985  0.0011953 ]
[ True False  True  True  True  True] 0.004276807390010993 [ 0.         -0.01088127 -0.00196692 -0.00292645

In [49]:
kinetics_df

,Experiment,max_conversion,reaction_end,squared_error,fit_p1,fit_p2,p1_err1,p1_err2,p2_err1,p2_err2
0,301_4_A_DMF,0.032374,yet to calc,0.000233,0.021583,14.416174,0.000015,-37554.463456,-37554.463456,4.844291e+14
1,302_4_A_DMSO,0.109489,yet to calc,0.000054,0.199226,0.052620,0.001595,-0.000578,-0.000578,2.130924e-04
2,316_10_E_DMSO,1.000000,yet to calc,0.071159,1.000000,0.356366,0.012476,-0.009427,-0.009427,1.357395e-02


In [66]:
# normalize the errors by dividing them by their respective standard deviation
def normalize_errors(errors):
    return errors / np.std(errors)

for error in ["squared_error", "p1_err1", "p1_err2", "p2_err1", "p2_err2"]:
    kinetics_df[error] = normalize_errors(kinetics_df[error])

kinetics_df

,Experiment,max_conversion,reaction_end,squared_error,fit_p1,fit_p2,p1_err1,p1_err2,p2_err1,p2_err2
0,0_6_A_DMF,0.516574,yet to calc,0.132619,0.735853,0.079203,4.153013e-06,-7.039353e-07,-7.039353e-07,9.499459e-18
1,1_6_A_DMSO,0.697321,yet to calc,0.065994,0.802498,0.133161,3.664366e-07,-1.139481e-07,-1.139481e-07,3.119301e-18
2,2_6_A_Tol,0.323281,yet to calc,0.019634,1.000000,0.025778,4.337425e-05,-1.333976e-06,-1.333976e-06,3.140197e-18
3,3_7_A_DMF,0.565125,yet to calc,0.050310,0.953088,0.059934,4.275014e-06,-3.874440e-07,-3.874440e-07,2.739075e-18
4,4_7_A_DMSO,0.790565,yet to calc,0.075483,1.000000,0.097675,1.877854e-06,-3.320342e-07,-3.320342e-07,4.762951e-18
...,...,...,...,...,...,...,...,...,...,...
312,312_9_E_DMF,0.734328,yet to calc,0.001451,0.796486,0.165271,4.090866e-09,-1.677918e-09,-1.677918e-09,6.627301e-20
313,313_9_E_DMSO,0.546547,yet to calc,0.000053,0.929680,0.204371,1.155602e-08,-3.678407e-09,-3.678407e-09,9.096412e-20
314,314_9_E_Tol,0.360000,yet to calc,0.142720,0.383237,0.197675,2.617283e-07,-2.749154e-07,-2.749154e-07,3.040122e-17
315,315_10_E_DMF,0.840426,yet to calc,0.200499,0.928939,0.154520,6.464905e-07,-2.092770e-07,-2.092770e-07,6.363344e-18


In [69]:
# drop conversions of less than 0.1 for error analysis
kinetics_df[kinetics_df["p1_err2"]>0.1]

,Experiment,max_conversion,reaction_end,squared_error,fit_p1,fit_p2,p1_err1,p1_err2,p2_err1,p2_err2
301,301_4_A_DMF,0.032374,yet to calc,0.025892,0.021583,14.416174,6.803620e-09,17.832974,17.832974,17.832643


In [96]:
# violin plots of the errors
kinetics_df_positive_errors = kinetics_df
for errors in ["p1_err1", "p1_err2", "p2_err1", "p2_err2"]:
    kinetics_df_positive_errors[errors] = kinetics_df_positive_errors[errors].apply(lambda x: np.absolute(x))
fig = px.violin(kinetics_df_positive_errors, y=["squared_error", "p1_err1", "p1_err2", "p2_err1", "p2_err2"], box=True, points="all", log_y=True)
fig.update_layout(title="Errors normalize by \u03C3", xaxis_title="Error type", yaxis_title="value", yaxis=dict(title="log(value)", range=(-22, 3)))
fig.show()


In [80]:
# plot entry 301 from the dataset
ik = 301
fit_data = kinetics_df.iloc[ik]
fig = px.line(title="Kinetic Curve Fit", labels={"x":"time", "y":"conversion"}); marker_dict = dict(color=colors[1%len(colors)])
fig.add_scatter(x=kinetic_curves[ik]["time"], y=kinetic_curves[ik]["conversion"], mode="lines+markers", name=kinetic_curves[ik]["experiment"][0], marker=marker_dict)
add_fits_to_plot(neg_growth, neg_growth_derivative, [fit_data["fit_p1"], fit_data["fit_p2"]], marker_dict)
fig.update_layout(yaxis=dict(range=[-0.1,1]), xaxis_title="Time [h]", yaxis_title="Conversion [%]")
fig.show()

[0.02158273314332938, 14.41617391397088]


C:\Users\fo72qev\AppData\Local\Temp\ipykernel_3744\52790876.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [28]:
# kinetics_df[kinetics_df["squared_error"]>0.01] 29/317 entries
# calculating covariance between errors to use only the reasonable ones

# permutate all combinations of the errors
errorcombs = [[],[],[]]
for err1, err2 in itertools.combinations(["squared_error", "p1_err1", "p1_err2", "p2_err1", "p2_err2"], 2):
    # print(f"The covariance between {err1} and {err2} is {np.cov(kinetics_df[err1], kinetics_df[err2])}")
    # print(f"The correlation between {err1} and {err2} is {np.correlate(kinetics_df[err1], kinetics_df[err2])}")
    # print("\n")
    errorcombs[0].append(f"{err1}/{err2}")
    errorcombs[1].append(np.cov(kinetics_df[err1], kinetics_df[err2]))
    errorcombs[2].append(*np.correlate(kinetics_df[err1], kinetics_df[err2]))
errorcombs_dc = {"name": errorcombs[0], "covariance":errorcombs[1], "correlation":errorcombs[2]}
errorcombs_df = pd.DataFrame(data=errorcombs_dc)
errorcombs_df["i_correlation"] = errorcombs_df["correlation"] * (-1)

# plot barplot per pair of errors with superimposed correlation
err_fig = px.bar(title="Correlation between Errors", labels={"correlation":"correlation"}, log_y=True)
err_fig.add_bar(x=errorcombs_df["name"], y=errorcombs_df["correlation"], name="postive correlation", marker_color="green")
err_fig.add_bar(x=errorcombs_df["name"], y=errorcombs_df["i_correlation"], name="negative correlation", marker_color="crimson")
err_fig.show()

In [29]:
# descry when a function aligns to the datapoints in a reasonable way
#   Hence, wheneth' the blunder exaggerates, an 80% betweeneth' of the maximum conversion in that kinetic should be assessed to be the maximum conversion.
